In [1]:
import pickle
import pandas as pd
import numpy as np
from numba import jit
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Dropout, Embedding, Input, MaxPooling2D, Conv2D, concatenate, Flatten, Reshape, BatchNormalization
from tqdm import tqdm
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD
import pickle
from sklearn.manifold import TSNE
import time
import random

# PreProcessing

In [ ]:
def add_negative_sample(path, saveas):
    data = pd.read_csv(path, names=['uid', 'lid', 'rate'])
    data = data[['uid', 'lid', 'rate']]
    users = data['uid'].unique()
    lists = data['lid'].unique()
    # data['rate'] = 1
    for u in users:
        i = 0
        while i < 5:
          l = random.choices(lists)
          if not (data[['uid', 'lid']].values == [u, l[0]]).all(axis=1).any():
                data.loc[len(data)] = [u, l[0], 0]
                i += 1
    data = data.sample(frac=1, random_state=1).reset_index(drop=True)
    data.to_csv(saveas, index=False)

In [ ]:
def split_data_train_test(path, name):
    test_percentage = 0.2
    df = pd.read_csv(path + name)
    df = df.sample(frac=1)

    df_train, df_test = train_test_split(
        df, test_size=test_percentage, random_state=1)


    df_train.to_csv(path + 'twitter_train.csv',
                    index=False)
    df_test.to_csv(path + 'twitter_test.csv',
                   index=False)

In [ ]:
def create_tokenizer(filename):
    # read all the tweets
    with open(Data_DIR + filename, 'rb') as f:
        tweets_dict = pickle.load(f)
    
    tweets_num = -np.inf
    for values in tweets_dict.values():
      if tweets_num < len(values):
        tweets_num = len(values)

    tokenizer = Tokenizer()
    for values in tweets_dict.values():
        tokenizer.fit_on_texts(values)

    vocab_size = len(tokenizer.word_index) + 1
    print("vocab size:", vocab_size)

    with open(Data_DIR + filename + '_tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    max_lens = []
    for values in tweets_dict.values():
        texts = tokenizer.texts_to_sequences(values)
        max_len_texts = max(len(x) for x in texts)
        max_lens.append(max_len_texts)

    max_len = max(max_lens)
    print("max len:", max_len)

    with open(Data_DIR + "datasetInfo.txt", "a") as f:
        f.write("vocab_size for " + filename + ": " + str(vocab_size))
        f.write("\n")
        f.write("max_len for " + filename + ": " + str(max_len))
        f.write("\n")
        f.write("tweets_num for " + filename + ": " + str(tweets_num))
        f.write("\n")
    f.close()

In [ ]:
def create_data(t, filepath):
  # read all the users tweets
  with open(Data_DIR + "user_tweets", 'rb') as f:
      user_tweets_dict = pickle.load(f)

  # read all the lists tweets
  with open(Data_DIR + "list_tweets", 'rb') as f:
      lists_tweets_dict = pickle.load(f)

  # read dataset maxlength and tweet_numbers
  with open(Data_DIR + "datasetInfo.txt", "r") as f:
          for line in f.readlines():
              name, value = line.split(": ")
              type, category = name.split(" for ")
              if type == "vocab_size":
                if category == "user_tweets":
                    vocab_size_users = int(value.strip("\n"))
                elif category == "list_tweets":
                    vocab_size_lists = int(value.strip("\n"))
              if type == "max_len":
                  if category == "user_tweets":
                      max_len_users = int(value.strip("\n"))
                  elif category == "list_tweets":
                      max_len_lists = int(value.strip("\n"))
              elif type == "tweets_num":
                  if category == "user_tweets":
                      tweet_number_users = 1000
                  elif category == "list_tweets":
                      tweet_number_lists = 500
  f.close()

  # read user_list_realtions
  df = pd.read_csv(filepath)
  print("len:", df.shape)

  uids = df['uid'].values

  # user tweets
  users_tweets = pd.DataFrame(columns=['user_id', 'tweets'])

  for id in uids:
      if id in user_tweets_dict.keys():
          # id wrote user_tweets_dict[id]
          users_tweets = pd.concat(
              [users_tweets, pd.DataFrame.from_records([{'user_id': id, 'tweets': user_tweets_dict[id]}])])

  lids = df['lid'].values
  # list tweets
  lists_tweets = pd.DataFrame(columns=['list_id', 'tweets'])

  for id in lids:
      if id in lists_tweets_dict.keys():
          # id wrote lists_tweets_dict[id]
          lists_tweets = pd.concat(
              [lists_tweets, pd.DataFrame.from_records([{'list_id': id, 'tweets': lists_tweets_dict[id]}])])

  # tokenizer
  with open(Data_DIR + 'user_tweets_tokenizer.pickle', 'rb') as handle:
      user_tokenizer = pickle.load(handle)

  with open(Data_DIR + 'list_tweets_tokenizer.pickle', 'rb') as handle:
      list_tokenizer = pickle.load(handle)

  user_trains = []
  for values in users_tweets['tweets']:
      user_train = user_tokenizer.texts_to_sequences(values)
      user_train = pad_sequences(
          user_train, padding='post', maxlen=max_len_users)
      user_trains.append(user_train)

  list_trains = []
  for values in lists_tweets['tweets']:
      list_train = list_tokenizer.texts_to_sequences(values)
      list_train = pad_sequences(
          list_train, padding='post', maxlen=max_len_lists)
      list_trains.append(list_train)

  Y = df['rate'].values
  with open( t + ".pkl", "wb") as f:
      pickle.dump([user_trains, list_trains, Y, df], f)
  f.close()

# Model

## Requirements

In [ ]:
# tokenizer

with open(Data_DIR + 'user_tweets_tokenizer.pickle', 'rb') as handle:
      user_tokenizer = pickle.load(handle)

with open(Data_DIR + 'list_tweets_tokenizer.pickle', 'rb') as handle:
    list_tokenizer = pickle.load(handle)

In [ ]:
with open(Data_DIR + "datasetInfo.txt", "r") as f:
        for line in f.readlines():
            name, value = line.split(": ")
            type, category = name.split(" for ")
            if type == "vocab_size":
              if category == "user_tweets":
                  vocab_size_users = int(value.strip("\n"))
              elif category == "list_tweets":
                  vocab_size_lists = int(value.strip("\n"))
            if type == "max_len":
                if category == "user_tweets":
                    max_len_users = int(value.strip("\n"))
                elif category == "list_tweets":
                    max_len_lists = int(value.strip("\n"))
            elif type == "tweets_num":
                if category == "user_tweets":
                    tweet_number_users = 1000
                elif category == "list_tweets":
                    tweet_number_lists = 500
f.close()

In [ ]:
embedding_size = 25
filter_sizes = [3, 4]
num_filters = 100
tweet_num_users = 1000
tweet_num_lists = 500
input_shape_u = (tweet_num_users, max_len_users)
input_shape_l = (tweet_num_lists, max_len_lists)

## GloVe Embedding

In [ ]:
embeddings_index = dict()
with open(Data_DIR + 'glove.twitter.27B/glove.twitter.27B.25d.txt', 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype=float)
        embeddings_index[word] = coefs
    f.close()

In [ ]:
# list embeddings_matrix
list_embeddings_matrix = np.zeros((vocab_size_lists, embedding_size))
for word, index in list_tokenizer.word_index.items():
    if index > vocab_size_lists - 1:
        break
    else:
        embeddings_vector = embeddings_index.get(word)
        if embeddings_vector is not None:
            list_embeddings_matrix[index] = embeddings_vector

# user embeddings_matrix
user_embeddings_matrix = np.zeros((vocab_size_users, embedding_size))
for word, index in user_tokenizer.word_index.items():
    if index > vocab_size_users - 1:
        break
    else:
        embeddings_vector = embeddings_index.get(word)
        if embeddings_vector is not None:
            user_embeddings_matrix[index] = embeddings_vector


## Networks

In [ ]:
from keras import layers, initializers, regularizers, constraints
import keras.backend as K
def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class FMLayer(layers.Layer):
  def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True,n_latent=32, fm_k=8, **kwargs):
    self.supports_masking = True
    self.init = initializers.get('glorot_uniform')

    self.W_regularizer = regularizers.get(W_regularizer)
    self.u_regularizer = regularizers.get(u_regularizer)

    self.W_constraint = constraints.get(W_constraint)
    self.u_constraint = constraints.get(u_constraint)

    self.n_latent = n_latent
    self.fm_k = fm_k
    super(FMLayer, self).__init__(**kwargs)

  def build(self, input_shape):

    self.W = self.add_weight(shape=(self.n_latent*2, 1),
                              initializer=self.init,
                              name='{}_W'.format(self.name),
                              regularizer=self.W_regularizer,
                              constraint=self.W_constraint,
                              trainable=True,)
    self.b = self.add_weight(
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     trainable=True,)

    self.u = self.add_weight(shape=(self.n_latent*2, self.fm_k),
                              initializer=self.init,
                              name='{}_u'.format(self.name),
                              regularizer=self.u_regularizer,
                              constraint=self.u_constraint,
                              trainable=True,)

    super(FMLayer, self).build(input_shape)

  def compute_mask(self, input, input_mask=None):
      # do not pass the mask to the next layers
      return None

  def call(self, x, mask=None):
      one = K.dot(x, self.W)
      inte1 = K.dot(x, self.u)
      inte2 = K.dot(K.square(x), K.square(self.u))  
      inter=(K.square(inte1)-inte2)*0.5
      inter = Dropout(0.5)(inter)
      inter=K.sum(inter,1,keepdims=True)
      return one + inter + self.b

  def compute_output_shape(self, input_shape):
      return input_shape[0], input_shape[-1]


In [ ]:
import keras
with tf.device('/device:GPU:0'):
  user_input = Input(shape=input_shape_u, dtype='int32')
  #Word embedding (using pre trained word vectors)
  embedder = Embedding(vocab_size_users, embedding_size, weights=[
                      user_embeddings_matrix],  input_length=tweet_num_users, trainable=True)
  user_embed = embedder(user_input)
  # print(user_embed.shape)
  pool_outputs = []
  for filter_size in filter_sizes:
    filter_shape = (filter_size, embedding_size)
    user_cnn = Conv2D(num_filters, filter_shape, strides=(1, 1), padding='valid',
                      data_format='channels_last', activation='relu',kernel_initializer='glorot_normal',
                      bias_initializer=keras.initializers.constant(0.1),
                      name='convolution_user_{:d}'.format(filter_size))(user_embed)
    max_pool_shape = (tweet_num_users - filter_size + 1, 1)
    user_pool = MaxPooling2D(pool_size=max_pool_shape,
                                      strides=(1, 1), padding='valid',
                                      data_format='channels_last',
                                      name='max_pooling_user_{:d}'.format(filter_size))(user_cnn)
    pool_outputs.append(user_pool)
  user_cnn = concatenate(pool_outputs, axis=-1)
  user_flat = Flatten()(user_cnn)
  user_drop = Dropout(0.5)(user_flat)
  user_output = Dense(258)(user_drop)

In [ ]:
with tf.device('/device:GPU:0'):
  list_input = Input(shape=input_shape_l, dtype='int32')
  #Word embedding (using pre trained word vectors)
  list_embedder = Embedding(vocab_size_lists, embedding_size, weights=[
      list_embeddings_matrix], input_length=tweet_num_lists, trainable=True)
  list_embed = list_embedder(list_input)
  pool_outputs = []
  for filter_size in filter_sizes:
    filter_shape = (filter_size, embedding_size)
    list_cnn = Conv2D(num_filters, filter_shape, strides=(1, 1), padding='valid',
                      data_format='channels_last', activation='relu',kernel_initializer='glorot_normal',
                      bias_initializer=keras.initializers.constant(0.1),
                      name='convolution_list_{:d}'.format(filter_size))(list_embed)
    max_pool_shape = (tweet_num_lists - filter_size + 1, 1)
    list_pool = MaxPooling2D(pool_size=max_pool_shape,
                                      name='max_pooling_list_{:d}'.format(filter_size))(list_cnn)
    pool_outputs.append(list_pool)
  list_cnn = concatenate(pool_outputs, axis=-1)
  list_flat = Flatten()(list_cnn)
  list_drop = Dropout(0.5)(list_flat)
  list_output = Dense(258)(list_drop)

In [ ]:
with tf.device('/device:GPU:0'):
  concat = concatenate([user_output, list_output], axis=-1)
  activate = tf.keras.layers.ReLU()(concat)
  main_output = FMLayer(n_latent=258)(activate)

In [ ]:
model = Model(inputs=[user_input, list_input], outputs=main_output)

## Train

In [ ]:
u, l, y, df = pd.read_pickle(Data_DIR + "train.pkl")
u = pad_sequences(u, padding='post', maxlen=tweet_num_users)
l = pad_sequences(l, padding='post', maxlen=tweet_num_lists)

In [ ]:
def loss_fn(y_true, y_pred):
    return K.sum((K.cast(y_true, K.floatx()) - y_pred)**2) / 2

def rmse(y_true, y_pred):
	return K.sqrt(K.mean(K.square(y_pred - K.cast(y_true, K.floatx())), axis=-1))
 
def mae(y_true, y_pred):
	return K.mean(K.abs(y_pred - K.cast(y_true, K.floatx())), axis=-1)

In [ ]:
model1 = Model(inputs=[user_input, list_input], outputs=main_output)
model1.compile(loss=loss_fn,
              optimizer=tf.keras.optimizers.RMSprop(), metrics=[rmse, mae, 'accuracy'])
early_stopping = EarlyStopping(monitor='rmse', patience=3, mode='max')
model1.fit([u, l], y, batch_size=100, callbacks=[early_stopping],
          epochs=10, validation_split=0.2)

Epoch 1/10
149/149 [==============================] - 286s 2s/step - loss: 5830375.0000 - rmse: 35.6318 - mae: 35.6318 - accuracy: 0.4974 - val_loss: 36.7141 - val_rmse: 0.7266 - val_mae: 0.7266 - val_accuracy: 0.3738
Epoch 2/10
149/149 [==============================] - 263s 2s/step - loss: 61.7370 - rmse: 0.6894 - mae: 0.6894 - accuracy: 0.5920 - val_loss: 11.7141 - val_rmse: 0.3842 - val_mae: 0.3842 - val_accuracy: 0.6583
Epoch 3/10
149/149 [==============================] - 262s 2s/step - loss: 11.8659 - rmse: 0.4060 - mae: 0.4060 - accuracy: 0.6546 - val_loss: 10.1032 - val_rmse: 0.3797 - val_mae: 0.3797 - val_accuracy: 0.6766
Epoch 4/10
149/149 [==============================] - 262s 2s/step - loss: 10.7831 - rmse: 0.3880 - mae: 0.3880 - accuracy: 0.6846 - val_loss: 9.9050 - val_rmse: 0.3897 - val_mae: 0.3897 - val_accuracy: 0.6796


In [ ]:
import matplotlib.pyplot as plt
def plot_history(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Valid'], loc='upper left')
    plt.show()

    plt.plot(history.history['rmse'])
    plt.plot(history.history['val_rmse'])
    plt.title('Model RMSE')
    plt.ylabel('RMSE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Valid'], loc='upper left')
    plt.show()

    plt.plot(history.history['mae'])
    plt.plot(history.history['val_mae'])
    plt.title('Model MAE')
    plt.ylabel('MAE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Valid'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Valid'], loc='upper left')
    plt.show()    


## Test

In [ ]:
data = df[['uid', 'lid', 'rate']]
users = data['uid'].unique()
lists = data['lid'].unique()
i = 6
max = len(users)
hits = []
while i < max:
  user = users[i]
  test = pd.DataFrame(columns=['uid', 'lid', 'rate'])
  for l in lists:
    if (data[['uid', 'lid', 'rate']].values == [user, l, 1]).all(axis=1).any():
      test.loc[len(test)] = [user, l, 1]
    else:
      test.loc[len(test)] = [user, l, 0]
  test = test.sample(frac=1).reset_index(drop=True)
  test.to_csv(Data_DIR + "Tests/twitter_test_" + str(user) + ".csv", index=False)
  create_data(Data_DIR + "test", 
            Data_DIR + "Tests/twitter_test_"+ str(user) +".csv")
  ut, lt, yt, dft = pd.read_pickle(Data_DIR + "test.pkl")
  if dft['uid'][0] != user:
    print("wrong id ... ", dft['uid'][0])
    break
  ut = pad_sequences(ut, padding='post', maxlen=tweet_num_users)
  lt = pad_sequences(lt, padding='post', maxlen=tweet_num_lists)
  predictions = model1.predict([ut, lt], batch_size=100, verbose=1)
  dft['prediction'] = np.array(predictions)
  sorted = dft.sort_values(by=['prediction'], ascending=False).reset_index(drop=True)
  sorted.to_csv(Data_DIR + "Predictions/output_" + str(user) + ".csv")
  indexs = sorted.loc[sorted['rate'] == 1].index
  count = 0
  for idx in indexs:
    if idx < 15 :
      count += 1
      hits.append(1)
    else:
      hits.append(0)
  recall = count / len(indexs)
  print(user, ", ", indexs, " ==> ", recall)
  i+=1